# name resolution

## I. python中的两类name和两种使用场景

### I.1 python中有两种形态的name，分别对应两套使用规则

  - <font color=blue>**unqualified name**</font>，形如：`x` 
    - 涉及除了用`.`表示的所有object name。包括：
      1. variable name of builtin data type
      2. function name
      3. module name
      4. class name
      - <font color=orange>注：这里class和module name不是他们的attribute name</font>
    - 做name resolution的时候适用LEGB lexical scoping rule
    - 分析的重点是看name所在namespace，以及由此决定的visibility scope。
  - <font color=blue>**qualified attribute name**</font>，形如：`object.x`
    - 涉及的attribute有三种
      1. class attribute：`class_name.attr_name`
      2. instance attribute: `instance_name.attr_name`
      3. module attribute: `module_name.attr_name`
      - <font color=orange>注：如果是直接的`class_name`和`module_name`，没有`.`，就是unqualified name。qualified对应的是attribute name</font>
    - 做name resolution的时候适用rules for modules and classes
    - 分析的重点是看类的继承关系决定的linked namespace tree。

### I.2 区分name assignment和name reference
- name的使用分两种：
  1. assignment会创建name或者修改name的value。
  2. reference只是调取使用name的value
- <font color=green>**name assignment**</font>时，python interpreter的处理对象是namespace。
  - <font color=red>**创建**</font>是根据name assignment发生的textual location，在对应的namespace中增加name和link (to the value object)。如果name assignment发生的textual location所对应的namespace中已经有同名name存在，那么此时的assignment会<font color=red>**修改**</font>这个找到的name的值。
- <font color=green>**name reference**</font>时，是根据reference发生的scope，先找当前scope中的namespace是否有该name，如果没有，就向上找。这里的“向上”有两种：
  - 一是<font color=blue>unqualified name</font>是根据LEGB rule，从当前scope向上找更高层的scope
  - 二是<font color=blue>qualified attribute name：</font>
    - <font color=orange>class-based object's attribute</font>是根据attribute inheritance关系，从当前instance或者class的namespace往superclass的namespace找
    - <font color=orange>module attribute</font>没有层，就只在module的当前namespace中找，没有找到就报错。
  - <font color=blue>**python也提供了跨层向上(只有向上，没有向下)做reference的方式：**</font>
    - <font color=blue>unqualified name</font>可以用namespace declaration来声明name为global或者nonlocal scope中的name，可以直接跳到对应的scope中的namespace查找name
    - <font color=blue>class-based qualified attribute name</font>可以直接将`object.name`中的`object`指定为想要search的superclass name，从而跳过比它低的class的namespace。

## I. unqualified name resolution
### I.1 适用于unqualified name的基本概念
#### I.1.1 namespace
- namespace是python按照name assignment发生的位置存放(name,object link)的地方。同一个space里面，没有重名name，从而保证了不会发生name collision。<font color=green>python还进一步用namespace和与之配套的规则来管理object的visibility和lifetime</font>
- 因为是按name assignment发生的location来把name存到不同namespace里面，因此需要有一个划分‘location’的规则，从而可以在每个独立的location unit中对应建立一个namespace。<font color=green>python的解决方式是将代码中的不同location划分成不同的scope。</font> 

#### I.1.2 scope
- scope是一个根据代码的文本结构来定义的概念。一共有四个layer，每个layer都对应着代码中的一块textual region。从下到上分别是local scope, enclosing scope, global scope和builtin scope。
| scope layer | textual location | 
|---------|---------|
| local scope   | 函数体内   | 
| enclosing scope   | enclosing function内，nested function外   | 
| global scope   | module内，函数和class之外  | 
| builtin scope   | builtin module的global scope   | 

- 每个scope layer中都有一个namespace。在这个scope中发生assignment的name就会存入或者修改对应namespace。
| name assignment location | 所属namespace | scope layer |
|---------|---------|---------|
| 函数体内 | function namespace   | local scope   |
| enclosing function内, nested function外   | enclosing function namespace   | enclosing scope   |
| module内，函数和class之外 | module namespace | global scope |
| builtin module的global scope  | builtin namespace | builtin scope |

- 如果想在local scope的代码中修改位于enclosing scope或者global scope中的name的值，直接assignment是不行的，但是可以用nonlocal和global declare的方式实现。

#### I.1.3 基于scope创建namespace的过程
- 从上面分析可见，存放unqualified name的namespace有三种：builtin namespace，global namespace和function namespace。
  - 在有嵌套关系的function中，enclosing function中的name会refer到nested function object上。他们还可能构建closure。<font color=brown>[详见function部分的notes]</font>但本质上他们都是function namespace。

- 三种namespace的创建和删除过程：
| namespace | 创建时间 | 删除时间 |
|---------|---------|---------|
| built-in namespace  | python interpreter启动时自动创建   | interpreter关闭时   |
| global namespace  | module imported or executed   | interpreter关闭时   |
| function namespace  | function call   | 函数退出时   |


- <font color=orange>namespace是一个dictionary object，scope是一个虚拟概念，并没有具体的python object与之对应。python通过namespace的创建方式使得namespace与scope layer联系起来：</font>
     - python interpreter在读到源代码module的时候就会对应建立builtin和module namespace，他们的文本scope自然就跟namespace对应了。
     - function被call的时候才会建立namespace，此时interpreter读取source code，函数体内如果有assignment，其name就放入namespace。‘函数体内’是一个textual location，因而也就跟scope关联起来。

### I.2 name visibility scope和LEGB lexical scoping rule

#### I.2.1 不同namespace中name的visibility scope
- visibility scope of a name: 是指在code的哪些scope范围内，可以直接access到该name。‘直接access’是指仅仅使用name就能refer到该name对应的object。
- <font color=blue>**一个name在源代码中assignment发生的textual location(对应的scope)决定了它所属的namespace，而它的namespace决定了它的visibility scope。**</font>

|层序号| 所属namespace| visibility scope |
|---------|---------|---------|
|1| function   | local scope   | 
|2| enclosing function   | enclosing scope (向下兼容1)   | 
|3| global   | global scope (向下兼容1-2)  |
|4| builtin   | builtin scope (向下兼容1-3)   |

- **向下兼容**：是指如果在layer更低(上面序号更小)的scope中做name reference，在低层中没有同名name导致的name collision的情况下，name可以直接被refer到。
- 上述visibility scope的实现，建立在python interpreter在做name reference的时候，采用了**LEGB rule**的基础上。

- 例如：两个概念的含义
  - x在function的namespace中: 指x定义在function的local scope中
  - x在function的scope中：指x要么在function的namespace中，要么在any of the outer namespaces that the function's namespace is currently nested within.

#### I.2.2 unqualified name reference的LEGB rule
- python interpreter在resolve name时会遵守LEGB rule:
  - Local: It first checks the local scope.
  - Enclosing: If not found, it checks any enclosing functions' scopes.
  - Global: Then, it checks the global scope.
  - Built-in: Finally, it checks the built-in scope.

## II. unqualified name resolution
-  这部分规则适用于class-based object和module
   - <font color=blue>**class based objects包括class和instance**</font>
- module很简单，参考module 部分notes。下面重点是说明class-besed object

### II.1 为什么unqualified name需要另一套resolution规则
- class和instance的namespace建立和name reference都不适用前面的规则。
- <font color=norange>为了方便地实现继承、多态等OOP特点，以及class相对于function和module所特有的multiple instance，customization inheritance等特征，python给class设计了它的namespace创建和使用规则。class主要做的是attribute name resolution，此时适用的规则不是看scope，而是search由类的继承关系决定的linked namespace tree。</font>

### II.2 class-based objects and module的namespace创建
1. class instance
   - <font color=brown>textual location</font>：不关心
   - <font color=brown>创建时间</font>：instance的namespace在instance创建的时候同步创建
2. class和module
   - <font color=brown>textual location</font>：不关心
   - <font color=brown>创建时间</font>：class和module的namespace是在他们被python interpreter读到的时候创建。也就是说，class statement被执行的时候，以及module被import的时候。

- <font color=blue>**这和function完全不同**</font>：
  1. python interpreter读到function的def statement的时候并不会创建namespace，之后function call的时候才创建。
  2. function的namespace层次取决于它在source code中的textual location。这也进一步决定了name的visibility scope。但是这里的namespace建立并不关心textual location，他们只关心object在inheritance tree上的位置。这个位置通过namespace创建时的在dictionary中加入的link来实现。

### II.3 reference rule
- class-based object从下到上，从左到右做inheritance tree search[详见class note 2]
- module直接search module namespace。

## III. class statement中的unqualified name resolution
- 一种特殊的场景是在class statement中做unqualified name resolution，上面两种情况都没有谈到。

### III.1 适用规则

- class statement在新建时，定义在其中的data attributes name和method attributes name都会计入class namespace。这与textual location无关。
- <font color=blue>**但如果method的定义中用了unqualified name，这种情况就要看scope。规则是：**</font>
  1. class的scope layer和function一样，都是local scope。
  2. method也和function一样，是local scope。
  3. 尽管method是nested in class的，但class不会像function那样因为嵌套而升级为enclosing scope。method中的name reference在search scope的时候会跳过class这一层。<font color=green>**即class和class中的top-level method都是LEGB中的L层，class不会升级成E层。如果method自己有nest，那么top-level method可能升级成nested function的E层。**</font>
- <font color=red>**因为class不能作为enclosing，所以method中如果要用到class attribute，无论是assignment还是reference，都只能用qualified name，用unqualified name无效。**</font>
     - 如果要用class attribute，直接用qualified name：
       1. 一般如果是要用所有instance share的class data attribute，用`class_name.attr_name`。
       2. 如果是调用别的method attribute，在method内部通常会用`self.method_name`
     - 如果method中要用global或者nonlocal(此时method嵌套在method里面，而不是class)，那么规则和unqualified name中一样，assignment要声明namespace。reference在没有name collision的情况下可以直接用，有collision的情况下也必须声明。为了代码可读性，这种情况最好都声明。

### III.2 典型例子

- **1. cqualified name可以继承class所有属性**
```python
class Person:
    numbers = 0             # class attribute
    def __init__(self, name):
        self.name = name    # 

bob = Person('bob')
john = Person('john')

print(Person.numbers)
print(bob.numbers)  # quanlified name继承了attribute

# output：
0
0
```

- **2. method的quanlified name使用qualified name规则**
```python
class Person:
    numbers = 0             # class attribute
    def __init__(self, name):
        # 这步同时发生了reference和assignment
        # reference发生了继承，赋值新建了instance attribute name
        self.numbers = self.numbers + 10  
        self.name = name

bob = Person('bob')
print(Person.numbers, bob.numbers)

# output：
0
10
```

- **3. 如果method中用unqualified name**
```python
class Person:
    numbers = 0             # class attribute
    def __init__(self, name):
        # numbers是unqualified name
        # class不构成enclosing scope，所以不能直接做reference
        self.numbers = numbers + 10 
        self.name = name 

bob = Person('bob')
print(Person.numbers, bob.numbers)

# output：
NameError: name 'numbers' is not defined
```

- **4. method中用unquanlified name时，适用LEGN rule**
```python
start_number = 0
end_number = 0

class Person:
    start_number = 0
    def __init__(self, name):
        Person.start_number += 1 # 必须指定class name，'Person'

        # assignment必须声明,reference可以不声明
        # 这里assignment end_number，reference start_number
        global end_number
        end_number = start_number + Person.start_number  
        
        self.name = name    # 关联self的都是instance attribute
        
bob = Person('bob')
john = Person('john')

print(start_number, end_number, Person.start_number)

# output:
0 2 2
```